In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive//content/gdrive/My Drive/Colab Notebooks/' 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#Import libs
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from fancyimpute import IterativeImputer
import pickle
import tensorflow as tf
print(tf.__version__)
#Using TensorFlow backend.#


ModuleNotFoundError: No module named 'sklearn.experimental'

In [4]:
#read in csv
raw_df = pd.read_csv('exercise_05_train_State_Farm.csv')

In [5]:
#Inspect data
raw_df.head()
#Without a data dictionary, using regular exploratory analysis will be less rigourous. 

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,0.963686,6.627185,-45.224008,9.477531,-3.216532,13.216874,9.754747,5.245851,-1.102918,-2.867482,...,0.988829,0.313772,asia,1.380664,-16.388994,5.326730,4.187294,0.045549,-3.646841,0
1,-1.770062,-23.610459,-0.964003,-31.981497,-10.294599,-10.240251,-1.518888,-1.675208,0.498134,-0.614390,...,-2.162863,1.809807,asia,2.500590,4.338834,-1.583225,-1.172417,0.011216,0.097180,0
2,9.962401,-8.349849,23.248891,-24.196879,8.937480,10.965000,-7.490596,-3.025094,0.595807,0.382732,...,1.779660,9.528113,asia,1.396475,7.839188,10.402396,1.288991,0.008209,-4.132316,0
3,-5.780709,-25.261584,1.383115,-11.786929,7.993078,-11.245752,-2.607351,-3.513896,-0.614235,-1.453979,...,-0.203206,4.892248,asia,0.744317,7.380982,7.599323,-8.022884,-0.067624,-1.796198,0
4,1.211541,1.119963,7.512938,21.987312,-5.155392,10.339416,3.045180,-0.619230,-0.928068,0.405024,...,0.248724,18.694990,asia,1.703196,-11.552129,0.381768,-3.550471,-0.055180,-3.344490,0


In [0]:
#Seperating categorical variables
category_df = raw_df.loc[:,raw_df.dtypes == np.object]
category_df.head()

,x34,x35,x41,x45,x68,x93
0,chrystler,thur,($865.28),0.02%,sept.,asia
1,volkswagon,thur,$325.27,-0.01%,July,asia
2,bmw,thurday,$743.91,0.00%,July,asia
3,nissan,thurday,$538.48,0.01%,July,asia
4,volkswagon,wed,($433.65),0.00%,Jun,asia


In [0]:

#Seperating numerical variables
num_df = raw_df.loc[:,raw_df.dtypes == np.float64]
num_df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x36,x37,x38,x39,x40,x42,...,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x94,x95,x96,x97,x98,x99
0,0.963686,6.627185,-45.224008,9.477531,-3.216532,13.216874,9.754747,5.245851,-1.102918,-2.867482,-37.632285,-12.983484,-30.244259,0.293407,-4.808540,9.293345,1.646353,5.687031,-14.049164,-1.158595,-20.776226,-0.789881,-39.396519,1.235763,-5.403411,-0.012328,0.703963,-22.946374,0.114717,-1.108137,-7.513845,1.278470,-3.606321,1.615670,-1.774488,8.804852,2.136329,2.501032,-6.011242,-32.366234,...,-40.332336,4.748333,-9.002899,7.083087,-12.264477,8.626495,0.065363,-0.124457,21.750625,-5.282121,6.153703,-6.272020,17.255821,5.782086,15.141420,33.884019,-2.389519,-1.766949,-17.897092,3.736479,40.082477,16.640042,1.680384,0.373888,-13.562891,13.158882,2.289092,-3.881315,-15.959124,2.940515,-2.555928,-37.601994,0.988829,0.313772,1.380664,-16.388994,5.326730,4.187294,0.045549,-3.646841
1,-1.770062,-23.610459,-0.964003,-31.981497,-10.294599,-10.240251,-1.518888,-1.675208,0.498134,-0.614390,47.652135,6.567264,-30.410216,-1.301751,-6.042590,1.817864,4.379207,5.676816,-37.757544,2.164646,59.052864,-1.480575,-35.736992,-8.580044,4.206910,0.732038,-1.109053,15.986872,-0.534066,5.795439,-2.556791,2.990327,-1.603091,0.633938,0.040507,6.645667,3.205207,3.567783,5.405315,-58.117719,...,-36.199657,-0.231633,-2.334688,-4.212509,1.039474,-7.251746,0.661180,-7.073426,-36.774709,-11.458640,10.030685,0.918319,-38.648511,-24.077239,16.991279,19.847963,1.106517,6.949901,15.688587,4.610910,-15.223066,4.880278,6.029540,-0.506606,-16.914889,-8.219612,1.639064,7.030400,-11.601869,-4.907436,-6.075066,-27.168761,-2.162863,1.809807,2.500590,4.338834,-1.583225,-1.172417,0.011216,0.097180
2,9.962401,-8.349849,23.248891,-24.196879,8.937480,10.965000,-7.490596,-3.025094,0.595807,0.382732,5.629537,3.769767,17.623199,-0.396468,-10.022546,1.158419,12.353464,0.398796,-0.201112,-5.803935,7.104037,2.393360,-30.622601,-12.318385,-6.341881,-0.055468,-1.176393,-13.288725,-0.383693,0.122918,-4.791641,-10.250248,5.837668,-3.756878,0.448073,-2.639292,-1.940933,-11.196415,-7.971329,-47.325525,...,-13.539267,2.706494,17.375989,-2.434721,-1.433001,7.734776,0.489224,0.535575,2.491412,-9.652969,16.090371,0.180281,10.326921,-12.845434,19.811216,-44.341947,-0.513749,-1.495533,3.605778,0.480531,39.328247,-19.626495,9.321199,1.036403,19.435797,-7.605212,-3.937989,3.702377,-2.246263,1.441523,-5.642358,-90.377656,1.779660,9.528113,1.396475,7.839188,10.402396,1.288991,0.008209,-4.132316
3,-5.780709,-25.261584,1.383115,-11.786929,7.993078,-11.245752,-2.607351,-3.513896,-0.614235,-1.453979,-2.636676,-3.595789,-19.648688,0.393391,-3.470142,0.102685,-0.882429,-1.598535,12.869959,5.374145,30.723272,4.541173,-59.270400,-0.101281,4.451334,0.650121,-1.154275,-19.434740,1.471404,9.678197,4.149206,-2.814893,-10.966831,6.516250,-0.625046,1.918934,1.046202,-6.727565,8.522306,-36.489186,...,-61.343401,-14.767857,9.686716,5.354954,0.002768,9.552721,-4.347074,1.513008,14.245827,1.555428,10.215072,5.658302,-39.728801,-22.905705,34.801687,-46.836184,-3.566841,-5.437052,-30.365259,0.494604,-6.350004,5.259606,0.073402,-0.174985,16.510493,1.141095,1.848256,-0.586822,2.902637,-2.598303,-1.431961,-57.211280,-0.203206,4.892248,0.744317,7.380982,7.599323,-8.022884,-0.067624,-1.796198
4,1.211541,1.119963,7.512938,21.987312,-5.155392,10.339416,3.045180,-0.619230,-0.928068,0.405024,-16.683612,6.534574,-39.182061,-0.424392,1.372472,-0.962540,0.269750,-0.303965,-9.051790,2.298528,-5.081549,1.782286,-5.537197,-0.048488,0.450021,0.121331,-0.677065,28.398174,0.108270,-0.159321,-9.267869,-2.649007,3.523005,2.563282,-0.739334,-2.391980,0.749308,-2.578158,8.913248,6.565492,...,-13.380797,-1.310366,14.936277,-14.064016,1.411826,-21.603894,-0.605150,0.796472,3.526055,0.437480,-10.153478,-5.454158,28.804591,28.42518

In [0]:
#summarize categorical variables
category_df.describe(include=[np.object])

,x34,x35,x68,x93
count,39992,39990,39991,39993
unique,10,5,12,3
top,volkswagon,wed,July,asia
freq,12622,20758,11114,35384


In [0]:
#convert categorical vars ($,%) to numeric
category_df['x41'] = category_df['x41'].str.replace('$','').astype(float)
category_df['x45'] = category_df['x45'].str.replace('%','').astype(float)

KeyError: ignored

In [0]:
#add converted vars to numeric df
num_df['x41'] = category_df['x41']
num_df['x45'] = category_df['x45']
category_df.drop(['x41','x45'],axis = 1, inplace = True)

KeyError: ignored

In [0]:
#print counts of each categorical variable
for col in category_df:
    print(category_df[col].value_counts(dropna=False),'\n')

volkswagon    12622
Toyota        10968
bmw            7262
Honda          5174
tesla          2247
chrystler      1191
nissan          326
ford            160
mercades         31
chevrolet        11
NaN               8
Name: x34, dtype: int64 

wed        20758
thurday    17752
tuesday      884
friday       543
monday        53
NaN           10
Name: x35, dtype: int64 

July       11114
Jun         9317
Aug         8170
May         4744
sept.       3504
Apr         1629
Oct          885
Mar          407
Nov          145
Feb           48
Dev           16
January       12
NaN            9
Name: x68, dtype: int64 

asia       35384
america     3167
euorpe      1442
NaN            7
Name: x93, dtype: int64 



In [0]:
#create mask to align same categories with different names
day_mask1 = category_df['x35'] == 'wednesday'
day_mask2 = category_df['x35'] == 'fri'
day_mask3 = category_df['x35']  == 'thur'

In [0]:
#align names
category_df['x35'][day_mask1] = 'wed'
category_df['x35'][day_mask2] = 'friday'
category_df['x35'][day_mask3] = 'thurday'

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [0]:
#perform inductive, single imputation with numerical df
#This imputation takes time to complete
imp = IterativeImputer()
imp_num_df = pd.DataFrame(imp.fit_transform(num_df))
imp_num_df.index = num_df.index
imp_num_df.columns = num_df.columns

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [0]:
#Save pickles in this file path
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks')
os.getcwd()

'/content/gdrive/My Drive/Colab Notebooks'

In [0]:
#Save imputation fit
with open('imp', 'wb') as i:
    pickle.dump(imp, i, pickle.HIGHEST_PROTOCOL)

In [0]:
#impute missing categorical vars as most frequent in category
for col in category_df:
    max_freq = category_df[col].value_counts().index[0]
    category_df[col][pd.isna(category_df[col])] = max_freq

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [0]:
#convert categories into nominal integers
os.chdir('/content/gdrive/My Drive/Colab Notebooks')
le_dict = {}
label_df = pd.DataFrame()
for col in category_df:
    le = LabelEncoder()
    label_df[col] = le.fit_transform(category_df[col])
    le_dict[col] = le

In [0]:
#save lable enc
with open('le_dict', 'wb') as l:
    pickle.dump(le_dict, l, pickle.HIGHEST_PROTOCOL)

In [0]:
#create df of binary cols representing instance of each category across multipile columns
enc = OneHotEncoder()
onehot_df = pd.DataFrame(enc.fit_transform(label_df).toarray())
#feature_names = enc.get_feature_names()
#onehot_df.columns = feature_names

In [0]:
#save one-hot-enc
with open('enc', 'wb') as e:
    pickle.dump(enc, e, pickle.HIGHEST_PROTOCOL)

In [0]:
scalar = StandardScaler()
scaled_num_df = pd.DataFrame(scalar.fit_transform(imp_num_df))

In [0]:
#save scalar
with open('scalar', 'wb') as s:
    pickle.dump(scalar, s, pickle.HIGHEST_PROTOCOL)

In [0]:
#create df of x vars from imputed df and encoded df
x_df = pd.concat([scaled_num_df, onehot_df], axis = 1)

In [0]:
#save x vars
with open('x_df', 'wb') as x:
    pickle.dump(x_df, x, pickle.HIGHEST_PROTOCOL)

In [0]:
#seperate dependant 'y' binary variable
y_df = raw_df['y'].astype('int')

In [0]:
#save y var
with open('y_df', 'wb') as y:
    pickle.dump(y_df, y, pickle.HIGHEST_PROTOCOL)